In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime, timedelta
import pytz

# === Initialize Firebase ===
if not firebase_admin._apps:
    cred = credentials.Certificate("serviceAccountKey.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()
tz = pytz.timezone("Africa/Mogadishu")
start = datetime.now(tz)
end = start + timedelta(minutes=20)

# === Function to get document ID by 'Name' with full debug ===
def get_doc_id_by_name(collection_ref, name):
    print(f"\n🔍 Searching for: {name}")
    docs = collection_ref.where("Name", "==", name).limit(1).stream()
    for doc in docs:
        print(f"✅ Found: {doc.id} → {doc.to_dict()}")
        return doc.id

    print("❌ No exact match found. Listing all documents in this collection:")
    all_docs = collection_ref.stream()
    for doc in all_docs:
        print(f" - {doc.id}: {doc.to_dict()}")
    raise Exception(f"\n❌ '{name}' not found in collection.\n")

# === Dynamic ID fetching ===
faculty_id = get_doc_id_by_name(db.collection("faculties"), "FOC")
department_id = get_doc_id_by_name(
    db.collection("faculties").document(faculty_id).collection("departments"), "CS"
)
class_id = get_doc_id_by_name(
    db.collection("faculties").document(faculty_id)
        .collection("departments").document(department_id)
        .collection("classes"),
    "BCS15B"
)

# Optional debug: List all available courses
print("\n📚 Listing all available courses before searching for DIP:")
course_collection = db.collection("faculties").document(faculty_id) \
    .collection("departments").document(department_id) \
    .collection("classes").document(class_id) \
    .collection("courses")

for doc in course_collection.stream():
    print(f" - {doc.id}: {doc.to_dict()}")

course_id = get_doc_id_by_name(course_collection, "CAT")

# === Add session document ===
session_ref = db.collection("faculties").document(faculty_id) \
    .collection("departments").document(department_id) \
    .collection("classes").document(class_id) \
    .collection("courses").document(course_id) \
    .collection("sessions")

session_ref.add({
    "subject": "CAT",
    "start_time": start,
    "end_time": end,
    "course_id": course_id,
    "class_id": class_id
})

print("\n✅ Session added successfully under dynamic structure.")



🔍 Searching for: FOC


C:\Users\abdirahman\Desktop\Face_Recognition\legacyenv\lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


✅ Found: AeRMJwkYXU0hFGRELx42 → {'Name': 'FOC'}

🔍 Searching for: CS
✅ Found: 8hROWXp4jP2hI1FOlUva → {'Name': 'CS'}

🔍 Searching for: BCS15B
✅ Found: 542nAO1705nO3HBEGWyH → {'Name': 'BCS15B'}

📚 Listing all available courses before searching for DIP:
 - 50a7SnMNqZ7x0R4YMhSa: {'Name': 'CAT'}
 - nfTNomGS7vMvjvC4ujXO: {'Name': 'AI'}
 - xEviw1UfoNDflkIhjaTK: {'Name': 'Ethics'}

🔍 Searching for: CAT
✅ Found: 50a7SnMNqZ7x0R4YMhSa → {'Name': 'CAT'}

✅ Session added successfully under dynamic structure.
